# Regression

Nicholas Cunningham  
CAP 5610, Final Exam

In [19]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

from sklearn.model_selection import KFold, train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.kernel_ridge import KernelRidge
from sklearn.metrics import mean_squared_error, r2_score

df = pd.read_csv('Exam Datasets\\regression_data.csv', header = 0)

X = df.iloc[:, 3:20].values
y = df.iloc[:, 2].values

### Preliminary Analysis

df.corr() and df.describe() were used to determine the correlation between each attribute in the dataset and general statistics about each attribute. The information was used to select which attributes to keep and which to discard. The data was processed without normalization to determine the performance on the raw attributes. A further analysis could determine whether normalization would improve the performance and by how much.


In [20]:
df.corr()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
id,1.000000,-0.016762,0.001286,0.005160,-0.012258,-0.132109,0.018525,-0.002721,0.011592,-0.023783,0.008130,-0.010842,-0.005151,0.021380,-0.016907,-0.008224,-0.001891,0.020799,-0.002901,-0.138798
price,-0.016762,1.000000,0.308350,0.525138,0.702035,0.089661,0.256794,0.266369,0.397293,0.036362,0.667434,0.605567,0.323816,0.054012,0.126434,-0.053203,0.307003,0.021626,0.585379,0.082447
bedrooms,0.001286,0.308350,1.000000,0.515884,0.576671,0.031703,0.175429,-0.006582,0.079532,0.028472,0.356967,0.477600,0.303093,0.154178,0.018841,-0.152668,-0.008931,0.129473,0.391638,0.029244
bathrooms,0.005160,0.525138,0.515884,1.000000,0.754665,0.087740,0.500653,0.063744,0.187737,-0.124982,0.664983,0.685342,0.283770,0.506019,0.050739,-0.203866,0.024573,0.223042,0.568634,0.087175
sqft_living,-0.012258,0.702035,0.576671,0.754665,1.000000,0.172826,0.353949,0.103818,0.284611,-0.058753,0.762704,0.876597,0.435043,0.318049,0.055363,-0.199430,0.052529,0.240223,0.756420,0.183286
sqft_lot,-0.132109,0.089661,0.031703,0.087740,0.172826,1.000000,-0.005201,0.021604,0.074710,-0.008958,0.113621,0.183512,0.015286,0.053080,0.007644,-0.129574,-0.085683,0.229521,0.144608,0.718557
floors,0.018525,0.256794,0.175429,0.500653,0.353949,-0.005201,1.000000,0.023698,0.029444,-0.263768,0.458183,0.523885,-0.245705,0.489319,0.006338,-0.059121,0.049614,0.125419,0.279885,-0.011269
waterfront,-0.002721,0.266369,-0.006582,0.063744,0.103818,0.021604,0.023698,1.000000,0.401857,0.016653,0.082775,0.072075,0.080588,-0.026161,0.092885,0.030285,-0.014274,-0.041910,0.086463,0.030703
view,0.011592,0.397293,0.079532,0.187737,0.284611,0.074710,0.029444,0.401857,1.000000,0.045990,0.251321,0.167649,0.276947,-0.053440,0.103917,0.084827,0.006157,-0.078400,0.280439,0.072575
condition,-0.023783,0.036362,0.028472,-0.124982,-0.058753,-0.008958,-0.263768,0.016653,0.045990,1.000000,-0.144674,-0.158214,0.174105,-0.361417,-0.060618,0.003026,-0.014941,-0.106500,-0.092824,-0.003406


In [38]:
df.describe()

,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
count,2.161300e+04,2.161300e+04,21613.000000,21613.000000,21613.000000,2.161300e+04,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000,21613.000000
mean,4.580302e+09,5.400881e+05,3.370842,2.114757,2079.899736,1.510697e+04,1.494309,0.007542,0.234303,3.409430,7.656873,1788.390691,291.509045,1971.005136,84.402258,98077.939805,47.560053,-122.213896,1986.552492,12768.455652
std,2.876566e+09,3.671272e+05,0.930062,0.770163,918.440897,4.142051e+04,0.539989,0.086517,0.766318,0.650743,1.175459,828.090978,442.575043,29.373411,401.679240,53.505026,0.138564,0.140828,685.391304,27304.179631
min,1.000102e+06,7.500000e+04,0.000000,0.000000,290.000000,5.200000e+02,1.000000,0.000000,0.000000,1.000000,1.000000,290.000000,0.000000,1900.000000,0.000000,98001.000000,47.155900,-122.519000,399.000000,651.000000
25%,2.123049e+09,3.219500e+05,3.000000,1.750000,1427.000000,5.040000e+03,1.000000,0.000000,0.000000,3.000000,7.000000,1190.000000,0.000000,1951.000000,0.000000,98033.000000,47.471000,-122.328000,1490.000000,5100.000000
50%,3.904930e+09,4.500000e+05,3.000000,2.250000,1910.000000,7.618000e+03,1.500000,0.000000,0.000000,3.000000,7.000000,1560.000000,0.000000,1975.000000,0.000000,98065.000000,47.571800,-122.230000,1840.000000,7620.000000
75%,7.308900e+09,6.450000e+05,4.000000,2.500000,2550.000000,1.068800e+04,2.000000,0.000000,0.000000,4.000000,8.000000,2210.000000,560.000000,1997.000000,0.000000,98118.000000,47.678000,-122.125000,2360.000000,10083.000000
max,9.900000e+09,7.700000e+06,33.000000,8.000000,13540.000000,1.651359e+06,3.500000,1.000000,4.000000,5.000000,13.000000,9410.000000,4820.000000,2015.000000,2015.000000,98199.000000,47.777600,-121.315000,6210.000000,871200.000000


## Split data into train and test set 80/20

The test set is partitioned off to determine a final performance metric for the 
best model found during training on new unseen, unlabeled data.

The remaining 80% of the data is used for training. 5-fold cross validation is 
performed on each model. The performance is measured using mean squared error 
and r2 on the validation set and the average performance across all 5 folds is 
calculated. Hyperparameters are tuned to improve the performance of each model.

In [21]:
# Split data into train and test set 80/20
X_Train, X_test, y_Train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=0)

## Analysis

ID and date are dropped from the attributes since they are highly uncorrelated 
from the price. Further attributes could be excluded due to lack of correlation 
(sqft_lot, condition, yr_built, zipcode, longitude) or due to high correlation 
with other variables (sqft_living15, sqft_lot15), but for the initial analysis 
most attributes were preserved to ensure no useful features were dropped. If 
more models and hyperparameters were trained on, the attributes would have to 
be pruned to speed up computation times.

5-fold cross validation is used to determine the performance of a model without 
sacrificing an additional proportion of the data to a validation set. k-fold 
cross validation is performed by splitting the training data in k equal 
partitions, then training the data on all but one of the partitions and 
determining the performance of the fold on the validation set. Each fold is 
trained in this way. The final performance is calculated from the mean of the 
performance on each fold.

GridSearchCV automatically performs hyperparameters selection and k-fold cross 
validation on a specified hyperparameter space. GridSearchCV outputs the best 
performing model in the hyperparameter space.

Mean squared error measures the sum of the (target - predicted) ^ 2. A lower 
mse is better, zero mse means the data was predicted perfectly.

r2 measures the variation between the target and predicted values. A value 
close to 1 indicates the predictions were close to the corresponding target. A 
value close to 0 indicates the model always predicts a value close to the 
sample mean. A negative value indicates the performance is worse than flipping 
a coin.

### Linear regression

In [22]:
# Linear regression
lr = LinearRegression()
mses = []
r2s = []

kf = KFold(n_splits=5, shuffle=True, random_state=0)
for train_index, validation_index in kf.split(X_Train):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    lr.fit(X_train, y_train)
    y_pred = lr.predict(X_validation)
    mse = mean_squared_error(y_validation, y_pred)
    r2 = r2_score(y_validation, y_pred)
    #print("Coefficients:\n", lr.coef_)
    mses.append(mse)
    r2s.append(r2)
    print("Mean squared error:", mse)
    print("r2 score:", r2)
print("\nmean mse:", np.mean(mses))
print("mean r2:", np.mean(r2s))

Mean squared error: 42291601576.0
r2 score: 0.687056548023
Mean squared error: 44126734512.9
r2 score: 0.684543852893
Mean squared error: 44728535039.1
r2 score: 0.691005376231
Mean squared error: 36630033755.9
r2 score: 0.714354890126
Mean squared error: 34375558508.5
r2 score: 0.71668476553

mean mse: 40430492678.5
mean r2: 0.698729086561


### Lasso regression, mean_squared_error

From eq. 3.29 in the book, lasso regression performs regularization with q=1 to 
reduce the magnitude of the coefficients in an attempt to prevent overfitting.

In [23]:
# Lasso regression, mean_squared_error
# regularization coeff: [0.001, 0.01, 0.1, 1, 2]
import pprint
pp = pprint.PrettyPrinter()

parameters = {'alpha' : [0.001, 0.01, 0.1, 1, 2]}
lasso = Lasso(random_state=0, max_iter=1000)

clf = GridSearchCV(lasso, parameters, cv=5, scoring='neg_mean_squared_error')
clf.fit(X_Train, y_Train)
#pp.pprint(clf.cv_results_)
print("\nBest Estimator:", clf.best_estimator_)
print("Best score: mean_squared_error =", clf.best_score_)
print("Best parameters:", clf.best_params_)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increa


Best Estimator: Lasso(alpha=2, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False)
Best score: mean_squared_error = -42254982541.4
Best parameters: {'alpha': 2}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### Lasso regression, r2_score

In [24]:
# Lasso regression, r2_score
# regularization coeff: [0.001, 0.01, 0.1, 1, 2]
parameters = {'alpha' : [0.001, 0.01, 0.1, 1, 2]}
lasso = Lasso(random_state=0, max_iter=1000)

clf = GridSearchCV(lasso, parameters, cv=5, scoring='r2', verbose=2)
clf.fit(X_Train, y_Train)
#pp.pprint(clf.cv_results_)
print("\nBest Estimator:", clf.best_estimator_)
print("Best score: r2_score =", clf.best_score_)
print("Best parameters:", clf.best_params_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
[CV] alpha=0.001 .....................................................
[CV] ...................................... alpha=0.001, total=   0.1s
[CV] alpha=0.001 .....................................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[CV] ...................................... alpha=0.001, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] ...................................... alpha=0.001, total=   0.1s
[CV] alpha=0.001 .....................................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[CV] ...................................... alpha=0.001, total=   0.1s
[CV] alpha=0.001 .....................................................
[CV] ...................................... alpha=0.001, total=   0.1s
[CV] alpha=0.01 ......................................................


...

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[CV] .......................................... alpha=2, total=   0.1s
[CV] alpha=2 .........................................................
[CV] .......................................... alpha=2, total=   0.1s
[CV] alpha=2 .........................................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


[CV] .......................................... alpha=2, total=   0.1s
[CV] alpha=2 .........................................................
[CV] .......................................... alpha=2, total=   0.1s
[CV] alpha=2 .........................................................
[CV] .......................................... alpha=2, total=   0.2s

Best Estimator: Lasso(alpha=2, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=0,
   selection='cyclic', tol=0.0001, warm_start=False)
Best score: r2_score = 0.695673188325
Best parameters: {'alpha': 2}


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  25 out of  25 | elapsed:    4.6s finished
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:484: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Fitting data with very small alpha may cause precision problems.
  ConvergenceWarning)


### Kernel ridge regression, mean_squared_error

Eq. 3.29 with q=2. More heavily penalized larger coefficients that lasso. 
Kernel methods are used to implicitly transform the input space from linear 
to polynomial of various degrees and to a gaussian space (rbf) with various 
gamma coefficients.

In [7]:
# Kernel ridge regression, mean_squared_error
krrParameters = [{'kernel':['linear']},
                 {'kernel':['poly'], 'degree':[2,4,7]},
                 {'kernel':['rbf'], 'gamma':[0.1,0.5,1,2,4]},
                 # coef0 = 1 # defaults to 1 for poly and sigmoid, else ignored
                ]

krr = KernelRidge()

clf2 = GridSearchCV(krr, krrParameters, cv=5, scoring='neg_mean_squared_error',
                   verbose=2)
clf2.fit(X_Train, y_Train)
#pp.pprint(clf2.cv_results_)
print("\nBest Estimator:", clf2.best_estimator_)
print("Best score: mean_squared_error =", clf2.best_score_)
print("Best parameters:", clf2.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.8s
[CV] kernel=linear ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.8s remaining:    0.0s


[CV] .................................... kernel=linear, total=  10.2s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.7s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=   9.9s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.0s
[CV] degree=2, kernel=poly ...........................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


[CV] ............................ degree=2, kernel=poly, total= 8.9min
[CV] degree=2, kernel=poly ...........................................


...

[CV] ............................ degree=7, kernel=poly, total= 8.8min
[CV] degree=7, kernel=poly ...........................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


[CV] ............................ degree=7, kernel=poly, total= 8.8min
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.1s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.0s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.1s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.2s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.5s
[CV] gamma=0.5, kernel=rbf ...........................................
[CV] ............................ gamma=0.5, kernel=rbf, total=  19.4s
[CV] gamma=0.5, kernel=rbf ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 413.7min finished



Best Estimator: KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None)
Best score: mean_squared_error = -42234850075.7
Best parameters: {'kernel': 'linear'}


### Kernel ridge regression, r2_score

In [10]:
# Kernel ridge regression, r2_score
krrParameters = [{'kernel':['linear']},
                 {'kernel':['poly'], 'degree':[2,4,7]},
                 {'kernel':['rbf'], 'gamma':[0.1,0.5,1,2,4]},
                 # coef0 = 1 # defaults to 1 for poly and sigmoid, else ignored
                ]

krr = KernelRidge()

clf3 = GridSearchCV(krr, krrParameters, cv=5, scoring='r2',
                   verbose=2)
clf3.fit(X_Train, y_Train)
#pp.pprint(clf3.cv_results_)
print("\nBest Estimator:", clf3.best_estimator_)
print("Best score: r2_score =", clf3.best_score_)
print("Best parameters:", clf3.best_params_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.2s
[CV] kernel=linear ...................................................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.1s remaining:    0.0s


[CV] .................................... kernel=linear, total=  10.0s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=   9.9s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.3s
[CV] kernel=linear ...................................................
[CV] .................................... kernel=linear, total=  10.1s
[CV] degree=2, kernel=poly ...........................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


[CV] ............................ degree=2, kernel=poly, total= 8.9min
[CV] degree=2, kernel=poly ...........................................


...

[CV] ............................ degree=7, kernel=poly, total= 8.8min
[CV] degree=7, kernel=poly ...........................................


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\ridge.py:154: UserWarning: Singular matrix in solving dual problem. Using least-squares solution instead.
  warnings.warn("Singular matrix in solving dual problem. Using "


[CV] ............................ degree=7, kernel=poly, total= 8.8min
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.1s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.2s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.0s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.0s
[CV] gamma=0.1, kernel=rbf ...........................................
[CV] ............................ gamma=0.1, kernel=rbf, total=  19.0s
[CV] gamma=0.5, kernel=rbf ...........................................
[CV] ............................ gamma=0.5, kernel=rbf, total=  19.1s
[CV] gamma=0.5, kernel=rbf ...........................................
[CV] .

[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed: 146.4min finished



Best Estimator: KernelRidge(alpha=1, coef0=1, degree=3, gamma=None, kernel='linear',
      kernel_params=None)
Best score: r2_score = 0.695825356018
Best parameters: {'kernel': 'linear'}


### Linear regression, reducing the number of attributes

Reducing the number of attributes very slightly improve performance. This shows that the parameters that were discarded did not contribute the price of the house.

In [37]:
# Linear regression, reducing the number of attributes

X_Train2 = X_Train[:,[0,1,2,4,5,6,8,9,10]]

lr2 = LinearRegression()
mses2 = []
r2s2 = []

kf2 = KFold(n_splits=5, shuffle=True, random_state=1)
for train_index, validation_index in kf2.split(X_Train2):
    X_train, X_validation = X[train_index], X[validation_index]
    y_train, y_validation = y[train_index], y[validation_index]
    
    lr2.fit(X_train, y_train)
    y_pred2 = lr2.predict(X_validation)
    mse2 = mean_squared_error(y_validation, y_pred2)
    r22 = r2_score(y_validation, y_pred2)
    #print("Coefficients:\n", lr.coef_)
    mses2.append(mse2)
    r2s2.append(r22)
    print("Mean squared error:", mse2)
    print("r2 score:", r22)
print("\nmean mse:", np.mean(mses2))
print("mean r2:", np.mean(r2s2))

Mean squared error: 40458284750.7
r2 score: 0.712504799624
Mean squared error: 41929138527.1
r2 score: 0.683166661448
Mean squared error: 39198535331.4
r2 score: 0.703518851339
Mean squared error: 39432512208.0
r2 score: 0.698847106942
Mean squared error: 40407963781.4
r2 score: 0.696693486873

mean mse: 40285286919.7
mean r2: 0.698946181245


# Results of Training
Linear  
mean mse: 40430492678.5  
mean r2: 0.698729086561

Lasso  
Best parameters: {'alpha': 2}  
Best score: mean_squared_error = -42254982541.4  
Best score: r2_score = 0.695673188325

Kernel Ridge  
Best parameters: {'kernel': 'linear'}  
Best score: mean_squared_error = -42234850075.7  
Best score: r2_score = 0.695825356018

Linear, reduced attributes  
mean mse: 40285286919.7  
mean r2: 0.698946181245

The linear model with the reduced number of attributes performed the best. This model was chosen to run the test set.

## Final Performance

In [47]:
y_pred = lr2.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
#print("Coefficients:\n", lr.coef_)

print("Mean squared error:", mse)
print("r2 score:", r2)

Mean squared error: 36315330765.0
r2 score: 0.694634821136
